In [36]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import html5lib
import numpy as np

In [37]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
tbody = soup.find("tbody")
rows = tbody.find_all("tr")

In [38]:
rows

[<tr>
 <th data-sort-type="number">Rank
 </th>
 <th>Bank name
 </th>
 <th>Market cap<br/>(US$ billion)
 </th></tr>,
 <tr>
 <td>1
 </td>
 <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635

In [39]:
data = []
bank_names = []
market_caps = []

for row in rows[1:]:  # Start from index 1 to skip the header row
    cols = row.find_all('td')
    bank_name = cols[1].text.strip()
    market_cap = cols[2].text.strip()
    bank_names.append(bank_name)
    market_caps.append(market_cap)

df = pd.DataFrame({
'Bank name': bank_names,
'MC_USD_Billion': market_caps
})
df['MC_USD_Billion'] = df['MC_USD_Billion'].apply(lambda x: float(x.replace('$', '').replace('B', '')))


In [40]:
df

,Bank name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [41]:
exchange_rate= pd.read_csv("exchange_rate.csv")
exchange_rate

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [42]:
exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']

In [43]:
df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

In [44]:
df

,Bank name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39
